In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [39]:
train = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_case4.csv')
test = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\test_case4.csv')
train_problem = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_train_problem.csv')
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

In [41]:
train = train.to_numpy()
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1

train_x = train
train_y = problem

In [42]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001262 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4699
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.793829	valid_0's pr_auc: 0.798678
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4860
[LightGBM] [Info] Numbe

In [43]:
print(np.mean(auc_scores))

0.7978115502603672


In [44]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,problem
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.444763,0.0,0.0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.444763,0.0,1.0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.405319,0.0,0.0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.444763,0.0,0.0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.444763,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.316156,-0.340504,-0.239957,-0.053688,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.491089,0.019092,0.482907,NaN,0.0,0.0
14996,-0.387084,-0.340504,-0.239957,-0.414143,-0.062749,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,NaN,0.0,0.0
14997,-0.078337,0.704060,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,0.115262,0.019092,0.764726,NaN,0.0,1.0
14998,-0.333983,-0.156620,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,-0.528213,-0.336841,-0.644371,NaN,0.0,1.0


In [45]:
clf = setup(data = train, target = "problem", silent=True, session_id=42) 

,Description,Value
0,session_id,42
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 64)"
5,Missing Values,True
6,Numeric Features,58
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [46]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7880,0.8131,0.4842,0.8149,0.6074,0.4747,0.5050,0.2710
lightgbm,Light Gradient Boosting Machine,0.7891,0.8082,0.5200,0.7850,0.6254,0.4870,0.5071,0.0480
rf,Random Forest Classifier,0.7861,0.8075,0.5008,0.7912,0.6132,0.4755,0.4994,0.1620
ada,Ada Boost Classifier,0.7837,0.8029,0.4955,0.7872,0.6081,0.4692,0.4932,0.0790
et,Extra Trees Classifier,0.7860,0.8015,0.4938,0.7976,0.6098,0.4734,0.4993,0.1750
lr,Logistic Regression,0.7726,0.7727,0.4260,0.8144,0.5592,0.4255,0.4662,0.1050
lda,Linear Discriminant Analysis,0.7595,0.7575,0.3605,0.8366,0.5036,0.3766,0.4348,0.0320
nb,Naive Bayes,0.7324,0.7406,0.3813,0.6972,0.4886,0.3288,0.3584,0.0160
knn,K Neighbors Classifier,0.7435,0.7265,0.4407,0.6901,0.5376,0.3720,0.3900,0.2290
dt,Decision Tree Classifier,0.6964,0.6653,0.5678,0.5510,0.5589,0.3277,0.3280,0.0430


In [47]:
blended = blend_models(estimator_list = best_3, optimize='AUC', method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7800,0.8019,0.4732,0.7925,0.5926,0.4547,0.4831
1,0.7886,0.8216,0.4958,0.8037,0.6132,0.4788,0.5052
2,0.7905,0.8029,0.5070,0.8000,0.6207,0.4858,0.5099
3,0.7962,0.8277,0.5309,0.8008,0.6385,0.5046,0.5253
4,0.7952,0.8033,0.4916,0.8373,0.6195,0.4921,0.5247
5,0.7790,0.7948,0.4747,0.7897,0.5930,0.4540,0.4817
6,0.7952,0.8250,0.5000,0.8279,0.6235,0.4944,0.5240
7,0.8000,0.8208,0.5169,0.8288,0.6367,0.5087,0.5357
8,0.7886,0.8276,0.5112,0.7913,0.6212,0.4838,0.5060


In [48]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7967,0.8068,0.4979,0.7910,0.6111,0.4830,0.5067


In [49]:
final_model = finalize_model(blended)

In [50]:
# pycaret의 predict_model을 사용하기 위해 test의 모든 컬럼을 정수로 변경
t_columns = list(np.arange(0,63))
test.columns = t_columns

In [51]:
prep_pipe = get_config('prep_pipe')
transformed_unseen_data = prep_pipe.transform(test)

In [52]:
prections = final_model.predict_proba(transformed_unseen_data)[:,1]
prections

array([0.94153247, 0.48923067, 0.66098552, ..., 0.74092222, 0.85650657,
       0.8598457 ])